# Econox Quick Start

This notebook demonstrates the basic workflow of solving a Dynamic Programming problem using **Econox**.

Econox is designed to build and estimate economic models by combining three core components: **Environment (Model)**, **Physics (Logic)**, and **Computation (Solver)**.

> **概要:** このノートブックでは、Econox を使用して動的計画法（Dynamic Programming）の問題を解く基本的な流れを紹介します。Econox は「環境（Model）」、「物理法則（Logic）」、「計算機（Solver）」を組み合わせてモデルを構築します。

We will solve a simple **Dynamic Discrete Choice Model** with the following settings:

* **Model**: 10 states, 3 actions.
* **Utility**: Linear utility function ($U = \beta x + \epsilon$).
* **Solver**: Value Iteration.

> ここでは、10状態・3行動のシンプルな環境で、線形効用関数を持つ動的離散選択モデルを「価値反復法（Value Iteration）」を用いて解きます。

## 0. Install & Import

First, install the library and import the necessary modules.

> まずはライブラリをインストールし、モジュールをインポートします。

In [ ]:
%pip install git+https://github.com/ito-haru/econox.git
%pip install jax

In [6]:
import jax
import jax.numpy as jnp
from econox import Model, LinearUtility, GumbelDistribution, ValueIterationSolver

## 1. Define the Environment (Model)

We define the "Environment" where the agents operate using `Model.from_data`. This container holds the state space size, action space size, and exogenous data (features, transition matrices).

> **1. 環境の定義**: 経済主体の置かれた環境を定義します。`Model.from_data` を使い、状態数、行動数、および外生的なデータ（特徴量や遷移確率）を格納したコンテナを作成します。

In [7]:
# Settings
num_states = 10
num_actions = 3
key = jax.random.PRNGKey(0)

# Dummy Data Generation
# Feature 'x': Random values for each state-action pair (shape: S, A, 1)
x_data = jax.random.normal(key, (num_states, num_actions, 1))

# Transition Matrix: Uniform random transitions (shape: S*A, S)
# In a real model, this would be a sparse matrix or specific transition logic.
transitions = jnp.ones((num_states * num_actions, num_states)) / num_states

# Create Model
model = Model.from_data(
    num_states=num_states,
    num_actions=num_actions,
    data={"x": x_data},
    transitions=transitions
)

print("Model created:", model)

Model created: Model(
  num_states=10, num_actions=3, data={'x': f64[10,3,1]}, transitions=f64[30,10]
)


## 2. Define the Physics (Logic)

Next, we define the "Physics" or rules governing the model.

* **Utility**: The objective function. Here, we use a linear utility on feature `x`.
* **Distribution**: The distribution of error terms. We use the Type I Extreme Value (Gumbel) distribution, which implies a Logit model.
* **Solver**: The algorithm to solve the model. We use Value Iteration with $\beta=0.95$.

> **2. ロジックの定義**: モデル内のルールを定義します。
> * **Utility**: 効用関数（特徴量 `x` に対する線形効用）。
> * **Distribution**: 誤差項の分布（ロジットモデルの仮定である Gumbel 分布）。
> * **Solver**: 解法（割引因子 $\beta=0.95$ の値反復法）。

In [8]:
# Define Utility Function
# U = param['beta'] * data['x']
utility = LinearUtility(param_keys=("beta",), feature_key="x")

# Define Error Distribution (Gumbel -> Logit Model)
dist = GumbelDistribution(scale=1.0)

# Define Solver
solver = ValueIterationSolver(discount_factor=0.95)

## 3. Solve the Model

We provide the parameters and solve the model.
Calling `solver.solve()` performs the fixed-point iteration to find the equilibrium (Value Function and Choice Probabilities).

> **3. モデルを解く**: パラメータを与えてモデルを解きます。`solver.solve()` を呼び出すと、内部で不動点反復が行われ、価値関数と選択確率が計算されます。

In [9]:
# Define Parameters
params = {"beta": jnp.array([1.0])}

# Solve
result = solver.solve(
    params=params,
    model=model,
    utility=utility,
    dist=dist
)

## 4. Check Results

The `SolverResult` object contains the solution (Value Function), the profile (Choice Probabilities), and convergence information.

> **4. 結果の確認**: `SolverResult` オブジェクトには、解（価値関数）、プロファイル（選択確率）、収束情報などが含まれています。

In [10]:
print(f"Convergence Success: {result.success}")
print(f"Number of Iterations: {result.aux['num_steps']}")

print("\n--- Value Function (First 5 states) ---")
print(result.solution[:5])

print("\n--- Choice Probabilities (First 5 states) ---")
print(result.profile[:5])

Convergence Success: True
Number of Iterations: 302

--- Value Function (First 5 states) ---
[30.58619926 29.6742716  30.33656094 29.83478006 31.29848729]

--- Choice Probabilities (First 5 states) ---
[[0.10972877 0.06150312 0.82876811]
 [0.40489404 0.36381637 0.2312896 ]
 [0.56887218 0.24277482 0.188353  ]
 [0.11951658 0.82039753 0.06008589]
 [0.095498   0.81886386 0.08563815]]
